In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data1 = pd.read_csv("C:/Users/Asus/Downloads/Divar.csv",low_memory=False,encoding='utf-8-sig') 

In [6]:
data = data1.copy()

In [9]:
drops = ['Unnamed: 0','city_slug','rent_value','rent_type','rent_credit_transform','transformable_rent','transformed_rent','neighborhood_slug','credit_value', 'transformable_credit','rent_to_single','location_radius','transformable_price', 'extra_person_capacity','title','rent_mode','price_mode','user_type','credit_mode','cat2_slug','cat3_slug']
data = data.drop(columns=drops,axis=1)
data

,created_at_month,description,price_value,transformed_credit,land_size,building_size,deed_type,has_business_deed,floor,rooms_count,...,has_sauna,floor_material,property_type,regular_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude
0,2024-08-01 00:00:00,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,NaN,NaN,NaN,500.0,NaN,NaN,NaN,سه,...,NaN,NaN,NaN,4.0,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600
1,2024-05-01 00:00:00,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,8.500000e+09,NaN,NaN,60.0,NaN,NaN,3,یک,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-10-01 00:00:00,تخلیه پایان ماه,NaN,NaN,NaN,132.0,NaN,NaN,3,سه,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459
3,2024-06-01 00:00:00,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,NaN,NaN,NaN,90.0,NaN,NaN,4,یک,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-05-01 00:00:00,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,5.750000e+09,NaN,NaN,115.0,single_page,NaN,4,دو,...,NaN,ceramic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2024-07-01 00:00:00,~~~مشاورین املاک قبادی~~~\n■جنوبی تک واحدی\n■د...,7.470000e+09,NaN,NaN,180.0,NaN,NaN,4.0,چهار,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.350235,47.083241
999996,2024-07-01 00:00:00,نوساز \n\n تک واحدی\n\nشخصی ساز\n\nروف گا...,NaN,3.000000e+09,NaN,110.0,NaN,NaN,1.0,دو,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.770454,51.369099
999997,2024-11-01 00:00:00,سلام ودرود\nفروش منزل مسکونی واقع در خیابان ان...,3.200000e+09,NaN,200.0,200.0,NaN,NaN,NaN,چهار,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999998,2024-09-01 00:00:00,سویت بدون خواب (روبه دریا و معمولی)\nسویت ۱خوا...,NaN,NaN,NaN,70.0,NaN,NaN,NaN,دو,...,NaN,NaN,NaN,5.0,NaN,3000000.0,NaN,NaN,37.483501,49.438721


In [11]:
data['area'] = data['description'].str.extract(r'(\d+)\s*متر')

In [13]:
def convert_number(x):
    
    if x == float('nan'):
        return x
    else:
        x = str(x)
        persian_digits ='۱۲۳۴۵۶۷۸۹۰١٢٣٤٥٦٧٨٩٠'
        english_digits ='12345678901234567890'
        translation_table = str.maketrans(persian_digits, english_digits)
        x = float(x.translate(translation_table))
        return x

In [15]:
data['area'] = data.area.apply(convert_number).astype('Int64')

In [17]:
data['land_size'] = data['land_size'].fillna(data.area)

In [19]:
data = data.drop(columns=['description','area'],axis=1)

In [21]:
def convert_material(x):
    if x == 'unselect':
        return float('nan')
    return x
data.floor_material = data.floor_material.apply(lambda x: convert_material(x))

In [23]:
def convert_unit(x):
    if x == 'more_than_8':
        return 9
    elif x == 'unselect':
        return float('nan')
    return float(x)
data.unit_per_floor = data.unit_per_floor.apply(lambda x: convert_unit(x))

In [25]:
def convert_total(x):
    if x == '30+':
        return 31
    elif x == 'unselect':
        return float('nan')
    return float(x)
data.total_floors_count = data.total_floors_count.apply(lambda x: convert_total(x))

In [27]:
def transform(u):
    if u == 'بدون اتاق':
        return int(0)
    elif u == 'یک':
        return int(1)
    elif u == 'دو':
        return int(2)
    elif u == 'سه':
        return int(3)
    elif u == 'چهار':
        return int(4)
    elif u == 'پنج یا بیشتر':
        return int(5)

In [29]:
data.rooms_count = data.rooms_count.apply(lambda x: transform(x))
data.rooms_count

0         3.0
1         1.0
2         3.0
3         1.0
4         2.0
         ... 
999995    4.0
999996    2.0
999997    4.0
999998    2.0
999999    1.0
Name: rooms_count, Length: 1000000, dtype: float64

In [31]:
data['month'] = pd.to_datetime(data['created_at_month']).dt.month
data['year'] = pd.to_datetime(data['created_at_month']).dt.year
data = data.drop('created_at_month',axis=1)
data

,price_value,transformed_credit,land_size,building_size,deed_type,has_business_deed,floor,rooms_count,total_floors_count,unit_per_floor,...,property_type,regular_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,month,year
0,NaN,NaN,500.0,500.0,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,4.0,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,8,2024
1,8.500000e+09,NaN,<NA>,60.0,NaN,NaN,3,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2024
2,NaN,NaN,<NA>,132.0,NaN,NaN,3,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,10,2024
3,NaN,NaN,90.0,90.0,NaN,NaN,4,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2024
4,5.750000e+09,NaN,<NA>,115.0,single_page,NaN,4,2.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,7.470000e+09,NaN,<NA>,180.0,NaN,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,34.350235,47.083241,7,2024
999996,NaN,3.000000e+09,30.0,110.0,NaN,NaN,1.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.770454,51.369099,7,2024
999997,3.200000e+09,NaN,200.0,200.0,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,2024
999998,NaN,NaN,<NA>,70.0,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,5.0,NaN,3000000.0,NaN,NaN,37.483501,49.438721,9,2024


In [33]:
def convert_construction_year(x):
    if x == 'قبل از ۱۳۷۰':
        return 1370
    elif x == float('nan'):
        return x
    else:
        x = str(x)
        persian_digits ='۱۲۳۴۵۶۷۸۹۰١٢٣٤٥٦٧٨٩٠'
        english_digits ='12345678901234567890'
        translation_table = str.maketrans(persian_digits, english_digits)
        x = float(x.translate(translation_table))
        return x

In [35]:
data['construction_year'] = data.construction_year.apply(convert_construction_year).astype('Int64')

In [37]:
data.floor.unique()

array([nan, '3', '4', '2', '1', '5', '0', '30', '6', '7', '8', '18', '12',
       '11', '10', '-1', '13', '9', '19', '16', '30+', '21', '15', '22',
       '14', '17', '20', '23', '25', '24', '26', '28', '29', '27', '3.0',
       '2.0', '4.0', '1.0', '0.0', '20.0', '5.0', '7.0', '6.0', '18.0',
       '8.0', '-1.0', '22.0', '10.0', '14.0', '9.0', '11.0'], dtype=object)

In [39]:
def convert_floor(x):
    if x == '30+':
        return 30
    return float(x)
data.floor = data.floor.apply(lambda x: convert_floor(x))

In [41]:
df = data.copy()

In [43]:
df = df[~(df['price_value'].isna()) | ~(df['transformed_credit'].isna())]

In [47]:
object_cols = df.select_dtypes(include=['object']).columns
object_cols

Index(['deed_type', 'has_business_deed', 'has_balcony', 'has_elevator',
       'has_warehouse', 'has_parking', 'is_rebuilt', 'has_water',
       'has_warm_water_provider', 'has_electricity', 'has_gas',
       'has_heating_system', 'has_cooling_system', 'has_restroom',
       'has_security_guard', 'has_barbecue', 'building_direction', 'has_pool',
       'has_jacuzzi', 'has_sauna', 'floor_material', 'property_type'],
      dtype='object')

In [49]:
df = pd.get_dummies(df, columns = object_cols,dtype = int, prefix_sep='_')
df

,price_value,transformed_credit,land_size,building_size,floor,rooms_count,total_floors_count,unit_per_floor,construction_year,regular_person_capacity,...,floor_material_floor_covering,floor_material_laminate_parquet,floor_material_mosaic,floor_material_stone,floor_material_wood_parquet,property_type_beach,property_type_jungle,property_type_jungle-mountain,property_type_mountain,property_type_other
1,8.500000e+09,NaN,<NA>,60.0,3.0,1.0,NaN,NaN,1384,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5.750000e+09,NaN,<NA>,115.0,4.0,2.0,6.0,NaN,1403,NaN,...,0,0,0,0,0,0,0,0,0,0
5,NaN,4.000000e+08,100.0,100.0,3.0,2.0,3.0,2.0,1389,NaN,...,0,0,0,0,0,0,0,0,0,0
7,8.700000e+09,NaN,100.0,100.0,4.0,2.0,NaN,NaN,1393,NaN,...,0,0,0,0,0,0,0,0,0,0
8,6.500000e+08,NaN,78.0,78.0,4.0,2.0,6.0,4.0,1396,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999992,4.350000e+09,NaN,<NA>,105.0,2.0,2.0,5.0,2.0,1401,NaN,...,0,0,0,0,0,0,0,0,0,0
999995,7.470000e+09,NaN,<NA>,180.0,4.0,4.0,NaN,NaN,1403,NaN,...,0,0,0,0,0,0,0,0,0,0
999996,NaN,3.000000e+09,30.0,110.0,1.0,2.0,NaN,NaN,1403,NaN,...,0,0,0,0,0,0,0,0,0,0
999997,3.200000e+09,NaN,200.0,200.0,NaN,4.0,NaN,NaN,1370,NaN,...,0,0,0,0,0,0,0,0,0,0


In [51]:
from sklearn.model_selection import train_test_split
target = ['price_value','transformed_credit']
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(target, axis=1), 
    df[target],
    test_size=0.3,
    random_state=42
)

In [92]:
xy_train_price = pd.concat([X_train,y_train.price_value],axis=1)
xy_train_credit = pd.concat([X_train,y_train.transformed_credit],axis=1)
xy_test_price = pd.concat([X_test,y_test.price_value],axis=1)
xy_test_credit = pd.concat([X_test,y_test.transformed_credit],axis=1)

# price_value

In [94]:
xy_train_price = xy_train_price[xy_train_price.price_value.notna()]
xy_test_price = xy_test_price[xy_test_price.price_value.notna()]

In [96]:
xy_train_price = xy_train_price[xy_train_price.unit_per_floor.notna()]
xy_test_price = xy_test_price[xy_test_price.unit_per_floor.notna()]

In [98]:
xy_train_price = xy_train_price[xy_train_price.land_size.notna()]
xy_test_price = xy_test_price[xy_test_price.land_size.notna()]

In [100]:
xy_train_price.floor = xy_train_price.floor.fillna(xy_train_price.floor.median())
xy_test_price.floor = xy_test_price.floor.fillna(xy_train_price.floor.median())
xy_train_price.total_floors_count = xy_train_price.total_floors_count.fillna(xy_train_price.total_floors_count.median())
xy_test_price.total_floors_count = xy_test_price.total_floors_count.fillna(xy_train_price.total_floors_count.median())

In [102]:
drops = ['regular_person_capacity','cost_per_extra_person','rent_price_on_regular_days','rent_price_on_special_days','rent_price_at_weekends']
xy_train_price = xy_train_price.drop(columns=drops,axis=1)
xy_test_price = xy_test_price.drop(columns=drops,axis=1)

In [106]:
xy_train_price = xy_train_price[xy_train_price['location_longitude'].notna()]
xy_test_price = xy_test_price[xy_test_price['location_longitude'].notna()]

In [108]:
xy_train_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58712 entries, 409529 to 206080
Data columns (total 87 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   land_size                             58712 non-null  Float64
 1   building_size                         58712 non-null  float64
 2   floor                                 58712 non-null  float64
 3   rooms_count                           58712 non-null  float64
 4   total_floors_count                    58712 non-null  float64
 5   unit_per_floor                        58712 non-null  float64
 6   construction_year                     58712 non-null  Int64  
 7   location_latitude                     58712 non-null  float64
 8   location_longitude                    58712 non-null  float64
 9   month                                 58712 non-null  int32  
 10  year                                  58712 non-null  int32  
 11  deed_type_mult

In [110]:
xy_test_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24975 entries, 255166 to 656411
Data columns (total 87 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   land_size                             24975 non-null  Float64
 1   building_size                         24975 non-null  float64
 2   floor                                 24975 non-null  float64
 3   rooms_count                           24975 non-null  float64
 4   total_floors_count                    24975 non-null  float64
 5   unit_per_floor                        24975 non-null  float64
 6   construction_year                     24975 non-null  Int64  
 7   location_latitude                     24975 non-null  float64
 8   location_longitude                    24975 non-null  float64
 9   month                                 24975 non-null  int32  
 10  year                                  24975 non-null  int32  
 11  deed_type_mult

# credit

In [116]:
xy_train_credit = xy_train_credit[xy_train_credit.transformed_credit.notna()]
xy_test_credit = xy_test_credit[xy_test_credit.transformed_credit.notna()]

In [120]:
xy_train_credit = xy_train_credit[xy_train_credit.unit_per_floor.notna()]
xy_test_credit = xy_test_credit[xy_test_credit.unit_per_floor.notna()]

In [124]:
drops = ['regular_person_capacity','cost_per_extra_person','rent_price_on_regular_days','rent_price_on_special_days','rent_price_at_weekends']
xy_train_credit = xy_train_credit.drop(columns=drops,axis=1)
xy_test_credit = xy_test_credit.drop(columns=drops,axis=1)

In [128]:
xy_train_credit = xy_train_credit[xy_train_credit['location_longitude'].notna()]
xy_test_credit = xy_test_credit[xy_test_credit['location_longitude'].notna()]

In [132]:
xy_train_credit = xy_train_credit.drop(columns='land_size',axis=1)
xy_test_credit = xy_test_credit.drop(columns='land_size',axis=1)

In [136]:
xy_train_credit.floor = xy_train_credit.floor.fillna(xy_train_credit.floor.median())
xy_test_credit.floor = xy_test_credit.floor.fillna(xy_test_credit.floor.median())
xy_train_credit.total_floors_count = xy_train_credit.total_floors_count.fillna(xy_train_credit.total_floors_count.median())
xy_test_credit.total_floors_count = xy_test_credit.total_floors_count.fillna(xy_test_credit.total_floors_count.median())

In [138]:
xy_train_credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19271 entries, 949122 to 172401
Data columns (total 86 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   building_size                         19271 non-null  float64
 1   floor                                 19271 non-null  float64
 2   rooms_count                           19271 non-null  float64
 3   total_floors_count                    19271 non-null  float64
 4   unit_per_floor                        19271 non-null  float64
 5   construction_year                     19271 non-null  Int64  
 6   location_latitude                     19271 non-null  float64
 7   location_longitude                    19271 non-null  float64
 8   month                                 19271 non-null  int32  
 9   year                                  19271 non-null  int32  
 10  deed_type_multi_page                  19271 non-null  int64  
 11  deed_type_othe

In [140]:
xy_test_credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8356 entries, 538914 to 995264
Data columns (total 86 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   building_size                         8356 non-null   float64
 1   floor                                 8356 non-null   float64
 2   rooms_count                           8356 non-null   float64
 3   total_floors_count                    8356 non-null   float64
 4   unit_per_floor                        8356 non-null   float64
 5   construction_year                     8356 non-null   Int64  
 6   location_latitude                     8356 non-null   float64
 7   location_longitude                    8356 non-null   float64
 8   month                                 8356 non-null   int32  
 9   year                                  8356 non-null   int32  
 10  deed_type_multi_page                  8356 non-null   int64  
 11  deed_type_other

# Feature engineering

In [143]:
corr_price = pd.DataFrame(xy_train_price.corr(method='spearman').price_value)

In [144]:
corr_price.price_value = corr_price.price_value.mask(corr_price.price_value<0,np.abs(corr_price.price_value))

In [145]:
corr_price.sort_values(by = 'price_value',ascending =  False).head(20)

,price_value
price_value,1.000000
building_size,0.568176
has_restroom_squat,0.524924
rooms_count,0.524705
has_restroom_squat_seat,0.483770
land_size,0.471195
floor_material_stone,0.429061
has_heating_system_heater,0.383026
has_elevator_False,0.375206
has_elevator_True,0.375124


In [146]:
feature_of_price = corr_price.sort_values(by = 'price_value',ascending =  False).index[0:15]

In [147]:
corr_credit = pd.DataFrame(xy_train_credit.corr(method='spearman').transformed_credit)

In [148]:
corr_credit.transformed_credit = corr_credit.transformed_credit.mask(corr_credit.transformed_credit<0,np.abs(corr_credit.transformed_credit))

In [155]:
corr_credit.sort_values(by = 'transformed_credit',ascending =  False).head(20)

,transformed_credit
transformed_credit,1.000000
building_size,0.573083
rooms_count,0.539561
has_restroom_squat,0.521628
has_restroom_squat_seat,0.480590
has_elevator_False,0.441322
has_elevator_True,0.440594
has_heating_system_heater,0.435362
has_warm_water_provider_water_heater,0.432649
floor_material_stone,0.410262


In [157]:
feature_of_credit = corr_credit.sort_values(by = 'transformed_credit',ascending =  False).index[0:16]

# Featuring

In [159]:
xy_train_price = xy_train_price[feature_of_price]
xy_test_price = xy_test_price[feature_of_price]
xy_train_credit = xy_train_credit[feature_of_credit]
xy_test_credit = xy_test_credit[feature_of_credit]

# Delete Outliers

In [196]:
xrp = xy_train_price.copy()
xrc = xy_train_credit.copy()

In [195]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

df01 = xrp.drop(axis=1,columns = ['price_value'])
df02 = xrc.drop(axis=1,columns = ['transformed_credit'])
scaler1 = StandardScaler()
scaler2 = StandardScaler()

df1 = scaler1.fit_transform(df01)
df2 = scaler2.fit_transform(df02)

model1 = DBSCAN(eps=1,min_samples=10)
model2 = DBSCAN(eps=10,min_samples=5)

clusters1 = model1.fit_predict(df1)
clusters2 = model2.fit_predict(df2)

s1 = scaler1.inverse_transform(df1)
s2 = scaler2.inverse_transform(df2)

xrp2 = pd.DataFrame(s1, columns=df01.columns)
xrc2 = pd.DataFrame(s2, columns=df02.columns)

b1 = (xrp['price_value'].reset_index())['price_value']
b2 = (xrc['transformed_credit'].reset_index())['transformed_credit']

xrp2 = pd.concat([xrp2,b1],axis =1)
xrc2 = pd.concat([xrc2,b2],axis =1)

xrp2 = xrp2[clusters1 !=-1]
xrc2 = xrc2[clusters2 !=-1]

# Random_forest for credit value

In [190]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import math

train_sample = xrc2.copy()


X_train = train_sample.drop('transformed_credit',axis=1)
y_train = train_sample.transformed_credit

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(max_samples=10000, n_jobs=-1,random_state = 42),
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error'
)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(xy_test_credit.drop('transformed_credit',axis=1))

print(r2_score(xy_test_credit.transformed_credit,y_pred))
print(mean_squared_error(xy_test_credit.transformed_credit,y_pred))
print(mean_absolute_error(xy_test_credit.transformed_credit,y_pred))

{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
0.562326193730009
6.996129956531752e+17
375005513.711807


# LightGBM for credit value

In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import joblib

train_sample = xrc2
X_train = train_sample.drop('transformed_credit',axis=1)
y_train = train_sample.transformed_credit

base_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',  
    'verbose': -1,
    'random_state': 42
}

param_grid = {
    'num_leaves': [15, 31, 63],
    'learning_rate': [0.01, 0.05, 0.1],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.7, 0.8, 0.9, 1],
    'min_data_in_leaf': [5, 10, 20, 40],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5]
}


model = lgb.LGBMRegressor(**base_params)


random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
print(random_search.best_params_)
print(-random_search.best_score_)
y_pred = best_model.predict(xy_test_credit.drop('transformed_credit',axis=1))
print(r2_score(xy_test_credit.transformed_credit,y_pred))
print(mean_squared_error(xy_test_credit.transformed_credit,y_pred))
print(mean_absolute_error(xy_test_credit.transformed_credit,y_pred))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'num_leaves': 31, 'min_data_in_leaf': 5, 'learning_rate': 0.05, 'lambda_l2': 0, 'lambda_l1': 0.5, 'feature_fraction': 0.6, 'bagging_fraction': 1}
827100642.3132266
0.5807010195786892
6.702411969015942e+17
371579746.2989397


# Random_Forest for price value

In [206]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

train_sample = xrp2.copy()

def conv(x):
    if x>0:
        return math.log10(x)
    return 0.001

def reconv(x):
    if x == 0.001:
        return 0
    return 10**x

train_sample['price_value'] = train_sample['price_value'].apply(lambda x: conv(x))
X_train = train_sample.drop('price_value',axis=1)
y_train = train_sample.price_value

param_grid = {
    'n_estimators': [200, 50],
    'max_depth': [5, 10],
    'min_samples_split': [20,100]
}
scoring = {
    
    
}
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(max_samples=20000, n_jobs=-1,random_state = 42),
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error'
)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(xy_test_price.drop('price_value',axis=1))
y_pred = pd.Series(y_pred).apply(lambda x: reconv(x))
print(r2_score(xy_test_price.price_value,y_pred))
print(mean_squared_error(xy_test_price.price_value,y_pred))
print(mean_absolute_error(xy_test_price.price_value,y_pred))

{'max_depth': 10, 'min_samples_split': 100, 'n_estimators': 200}
0.0008049319291737467
2.259225311327472e+23
13221774156.213041


# LightGBM for price value

In [205]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import joblib

train_sample = xrp2
X_train = train_sample.drop('price_value',axis=1)
y_train = train_sample.price_value

base_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',  
    'verbose': -1,
    'random_state': 42
}

param_grid = {
    'num_leaves': [15, 31, 63],
    'learning_rate': [0.01, 0.05, 0.1],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.7, 0.8, 0.9, 1],
    'min_data_in_leaf': [5, 10, 20, 40],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5]
}


model = lgb.LGBMRegressor(**base_params)


random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
print(random_search.best_params_)
print(-random_search.best_score_)

y_pred = best_model.predict(xy_test_price.drop('price_value',axis=1))
print(r2_score(xy_test_price.price_value,y_pred))
print(mean_squared_error(xy_test_price.price_value,y_pred))
print(mean_absolute_error(xy_test_price.price_value,y_pred))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'num_leaves': 63, 'min_data_in_leaf': 40, 'learning_rate': 0.01, 'lambda_l2': 0.5, 'lambda_l1': 0.1, 'feature_fraction': 0.9, 'bagging_fraction': 1}
639763493915.1992
-0.01955291718015051
2.3052553303514553e+23
27361408727.444355
